In [1]:
## Calling required libraries

In [2]:
import pandas as pd
import pickle
import os

In [3]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [4]:
## Loading the data

In [5]:
pickle_in = open("RAW DATA.pickle","rb")
Base_Data = pickle.load(pickle_in)
pickle_in.close()

In [6]:
### Getting NULL values for categorical variables

In [7]:
print(Base_Data.columns)

Index(['CLAIM_NBR', 'LINE_ITEM_CURRENT_ORIGINAL', 'LINE_ITEM_REWORK_ADD_PAY',
       'LINE_ITEM_OTHER_INSURANCE', 'LINE_ITEM_ORIGINAL_BACKED_OUT',
       'LINE_ITEM_REVERSAL_LINE', 'LINE_ITEM_HISTORY_LINE', 'LINE_ITEM_OTHERS',
       'PROCESSED_GROUP', 'CIS_CAT_MISSING',
       ...
       'MBR_RSP', 'DEDUC', 'MBR_ST', 'MBR_AGE', 'PRIM_DX_MAPPING', 'DIAG_CNT',
       'MSL_INDICATOR', 'REFERRAL', 'SUSDUP', 'AUDIT_STATUS'],
      dtype='object', length=211)


In [8]:
## Removing unrequired columns

In [9]:
## Getting numeric/categorical columns

In [10]:
print(Base_Data.columns)

Num_Cols = Base_Data.select_dtypes(include='number')
char_cols = Base_Data.select_dtypes(include='string')

print(char_cols.dtypes)
print(Num_Cols.dtypes)

Index(['CLAIM_NBR', 'LINE_ITEM_CURRENT_ORIGINAL', 'LINE_ITEM_REWORK_ADD_PAY',
       'LINE_ITEM_OTHER_INSURANCE', 'LINE_ITEM_ORIGINAL_BACKED_OUT',
       'LINE_ITEM_REVERSAL_LINE', 'LINE_ITEM_HISTORY_LINE', 'LINE_ITEM_OTHERS',
       'PROCESSED_GROUP', 'CIS_CAT_MISSING',
       ...
       'MBR_RSP', 'DEDUC', 'MBR_ST', 'MBR_AGE', 'PRIM_DX_MAPPING', 'DIAG_CNT',
       'MSL_INDICATOR', 'REFERRAL', 'SUSDUP', 'AUDIT_STATUS'],
      dtype='object', length=211)
PROCESSED_GROUP    string
PRCSD_AS           string
CLI_BFT            string
LOB                string
NTWK               string
PRV_SPC            string
TOB                string
PAR_CD             string
VCH                string
MBR_ST             string
PRIM_DX_MAPPING    string
MSL_INDICATOR      string
REFERRAL           string
SUSDUP             string
AUDIT_STATUS       string
dtype: object
CLAIM_NBR                                               Int64
LINE_ITEM_CURRENT_ORIGINAL                              Int64
LINE_ITEM_REW

In [11]:
## getting number of service days in SERV_DAYS column

In [12]:
##### categorical columns stored as numeric

  #All REV , CPT and REJ (These cols are already dummified)
  #CLAIM_NBR
  
###Factor columns (combining the categorical variables in the Num_cols except the REV , CPT and REJ)

In [13]:
Fact_cols = pd.concat([char_cols,Num_Cols[['CLAIM_NBR']]], axis=1)

In [14]:
### Getting the NULL %age for each column in Fact_cols table

In [15]:
x = Fact_cols.columns
for i in x:
    print(i,":", Fact_cols[i].isnull().sum()/len(Fact_cols["CLAIM_NBR"])*100)

PROCESSED_GROUP : 0.0
PRCSD_AS : 0.0
CLI_BFT : 0.0
LOB : 0.0
NTWK : 0.0
PRV_SPC : 0.007095385634209219
TOB : 51.273621721340554
PAR_CD : 17.147181949338947
VCH : 1.21094581490504
MBR_ST : 0.0
PRIM_DX_MAPPING : 1.1352617014734752
MSL_INDICATOR : 95.5630188500745
REFERRAL : 81.98954613183227
SUSDUP : 91.15441924268583
AUDIT_STATUS : 0.0
CLAIM_NBR : 0.0


In [16]:
## There are 6 columns with missing values ( TOB,CAS_CAUSE,SUSP,PRIMARY_LEVEL_2,EXCode,EDIT)

#### Getting claims' distribution for various categorical variables 

####################################################

### Function to copy data on clipboard

In [17]:
def write_to_excel(x,index=False,header=True):
    x.to_clipboard(excel=False,index=index,header=header)

In [18]:
############################
##Clubbing the minority class of Categorical variables into 'OTHERS'
############################

In [19]:
#1 PRCSD_AS

In [20]:
grp = Fact_cols.groupby('PRCSD_AS')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [21]:
###### Replacing minor level+NULLS PRCSD_AS with 'OTHERS'

In [22]:
Fact_cols['PRCSD_AS'] = Fact_cols['PRCSD_AS'].fillna('MISSING')

In [23]:
PRCSD_AS = pd.DataFrame(Fact_cols['PRCSD_AS'])
PRCSD_AS.iloc[(PRCSD_AS['PRCSD_AS'] != 'PAID') & (PRCSD_AS['PRCSD_AS'] != 'DENIED') & (PRCSD_AS['PRCSD_AS'] != 'HISTORY') & (PRCSD_AS['PRCSD_AS'] != 'PREPAID'), 'PRCSD_AS'] = 'OTHERS'

print(PRCSD_AS['PRCSD_AS'].value_counts())

DENIED     24478
PAID       16895
PREPAID      790
HISTORY      118
Name: PRCSD_AS, dtype: Int64


In [24]:
#2 CLI_BFT

In [25]:
grp = Fact_cols.groupby('CLI_BFT')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [26]:
##  CLI_BFT replacing NULLS with MISSING factor

In [27]:
Fact_cols['CLI_BFT'] = Fact_cols['CLI_BFT'].fillna('MISSING')

In [28]:
###### Replacing minor level+NULLS PI with 'OTHERS'

In [29]:
CLI_BFT = pd.DataFrame(Fact_cols['CLI_BFT'])
CLI_BFT.iloc[(CLI_BFT['CLI_BFT'] != '1') & (CLI_BFT['CLI_BFT'] != '3') & (CLI_BFT['CLI_BFT'] != '2')
             & (CLI_BFT['CLI_BFT'] != '4') & (CLI_BFT['CLI_BFT'] != '8') & (CLI_BFT['CLI_BFT'] != '5'), 'CLI_BFT'] = 'OTHERS'

print(CLI_BFT['CLI_BFT'].value_counts())

1         36722
3          2383
2          1741
4           600
8           299
5           297
OTHERS      239
Name: CLI_BFT, dtype: Int64


In [30]:
#3 LOB

In [31]:
grp = Fact_cols.groupby('LOB')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [32]:
##  GROUP_NAME replacing NULLS with MISSING factor

In [33]:
Fact_cols['LOB'] = Fact_cols['LOB'].fillna('MISSING')

In [34]:
###### Replacing minor level+NULLS GROUP_NAME with 'OTHERS'

In [35]:
LOB = pd.DataFrame(Fact_cols['LOB'])
LOB.iloc[(LOB['LOB'] != '3I') & (LOB['LOB'] != '32') & (LOB['LOB'] != '93')
                & (LOB['LOB'] != '1M') & (LOB['LOB'] != '39') & (LOB['LOB'] != '62') 
                & (LOB['LOB'] != '3G'), 'LOB'] = 'OTHERS'

print(LOB['LOB'].value_counts())

3I    20220
32    12137
93     3951
39     2168
62     1908
1M     1592
3G      305
Name: LOB, dtype: Int64


In [36]:
#4 NTWK

In [37]:
grp = Fact_cols.groupby('NTWK')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [38]:
##  NTWK replacing NULLS with MISSING factor

In [39]:
Fact_cols['NTWK'] = Fact_cols['NTWK'].fillna('MISSING')

In [40]:
###### Replacing minor level+NULLS GROUP_NAME with 'OTHERS'

In [41]:
NTWK = pd.DataFrame(Fact_cols['NTWK'])
NTWK.iloc[(NTWK['NTWK'] != '0000D3') & (NTWK['NTWK'] != '5241H3I') & (NTWK['NTWK'] != '7030H3I')
                & (NTWK['NTWK'] != '0241D3') & (NTWK['NTWK'] != '0000H32') & (NTWK['NTWK'] != '0784D9')
                & (NTWK['NTWK'] != '4030D3') & (NTWK['NTWK'] != '0000D1') & (NTWK['NTWK'] != '0000D6')
                & (NTWK['NTWK'] != '5367H3I') & (NTWK['NTWK'] != '7037H32') & (NTWK['NTWK'] != '0000D9')
                & (NTWK['NTWK'] != '4037D3') & (NTWK['NTWK'] != '0000H62') & (NTWK['NTWK'] != '0000H39')
                & (NTWK['NTWK'] != '5331H32') & (NTWK['NTWK'] != '5321H32') & (NTWK['NTWK'] != '5329H3I')
                & (NTWK['NTWK'] != '7034H3I') & (NTWK['NTWK'] != '5366H32') & (NTWK['NTWK'] != '7033H3I')
                & (NTWK['NTWK'] != '6031H3I') & (NTWK['NTWK'] != '5317H3I')
                & (NTWK['NTWK'] != '0785D9') & (NTWK['NTWK'] != '7037H39')
                , 'NTWK'] = 'OTHERS'

print(NTWK['NTWK'].value_counts())

0000D3     7885
5241H3I    4644
7030H3I    4154
0241D3     3730
OTHERS     3242
0000H32    2837
0784D9     2337
4030D3     1618
0000D1     1502
0000D6     1208
0000D9     1128
7037H32    1122
5367H3I    1002
4037D3      856
0000H62     576
0000H39     536
5329H3I     530
5321H32     518
5331H32     473
7033H3I     464
5366H32     401
7034H3I     399
5317H3I     364
6031H3I     285
0785D9      239
7037H39     231
Name: NTWK, dtype: Int64


In [42]:
#5 PRV_SPC

In [43]:
grp = Fact_cols.groupby('PRV_SPC')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [44]:
##  PRV_SPC replacing NULLS with MISSING factor

In [45]:
Fact_cols['PRV_SPC'] = Fact_cols['PRV_SPC'].fillna('MISSING')

In [46]:
###### Replacing minor level+NULLS PRV_SPC with 'OTHERS'

In [47]:
PRV_SPC = pd.DataFrame(Fact_cols['PRV_SPC'])
PRV_SPC.iloc[
                (PRV_SPC['PRV_SPC'] != 'U9')
                & (PRV_SPC['PRV_SPC'] != 'G5') & (PRV_SPC['PRV_SPC'] != '48') & (PRV_SPC['PRV_SPC'] != '04')
                & (PRV_SPC['PRV_SPC'] != 'A2') & (PRV_SPC['PRV_SPC'] != '38') & (PRV_SPC['PRV_SPC'] != '42')
                & (PRV_SPC['PRV_SPC'] != '55') & (PRV_SPC['PRV_SPC'] != 'R4') & (PRV_SPC['PRV_SPC'] != '49')
                & (PRV_SPC['PRV_SPC'] != '06') & (PRV_SPC['PRV_SPC'] != '61') & (PRV_SPC['PRV_SPC'] != '60')
                & (PRV_SPC['PRV_SPC'] != 'A1') & (PRV_SPC['PRV_SPC'] != 'B4') & (PRV_SPC['PRV_SPC'] != '63')
                & (PRV_SPC['PRV_SPC'] != '85') & (PRV_SPC['PRV_SPC'] != '27') & (PRV_SPC['PRV_SPC'] != '50')
                & (PRV_SPC['PRV_SPC'] != '08') & (PRV_SPC['PRV_SPC'] != '51') & (PRV_SPC['PRV_SPC'] != '59')
                & (PRV_SPC['PRV_SPC'] != 'A0') & (PRV_SPC['PRV_SPC'] != '20') & (PRV_SPC['PRV_SPC'] != '78')
                & (PRV_SPC['PRV_SPC'] != 'OA') & (PRV_SPC['PRV_SPC'] != '80') & (PRV_SPC['PRV_SPC'] != '26')
                & (PRV_SPC['PRV_SPC'] != '11') & (PRV_SPC['PRV_SPC'] != '47')
                & (PRV_SPC['PRV_SPC'] != '53') & (PRV_SPC['PRV_SPC'] != '01') & (PRV_SPC['PRV_SPC'] != '34'), 'PRV_SPC'] = 'OTHERS'

print(PRV_SPC['PRV_SPC'].value_counts())

G5        8764
OTHERS    5872
A2        3458
55        3207
06        3201
A1        2074
85        1127
08        1066
A0        1032
OA         998
11         900
53         854
48         696
38         646
R4         622
61         599
B4         593
27         551
51         533
20         525
80         517
47         495
01         465
04         460
42         411
49         383
60         327
63         303
50         300
59         290
78         289
26         258
34         249
U9         216
Name: PRV_SPC, dtype: Int64


In [48]:
#6 TOB

In [49]:
grp = Fact_cols.groupby('TOB')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [50]:
##  TOB replacing NULLS with MISSING factor

In [51]:
Fact_cols['TOB'] = Fact_cols['TOB'].fillna('MISSING')

In [52]:
###### Replacing minor level+NULLS TOB with 'OTHERS'

In [53]:
TOB = pd.DataFrame(Fact_cols['TOB'])
TOB.iloc[(TOB['TOB'] != 'MISSING')
                & (TOB['TOB'] != '131') & (TOB['TOB'] != '851') & (TOB['TOB'] != '771')
                & (TOB['TOB'] != '329') & (TOB['TOB'] != '711') & (TOB['TOB'] != '141')
                & (TOB['TOB'] != '323') & (TOB['TOB'] != '831') & (TOB['TOB'] != '111')
                & (TOB['TOB'] != '322') & (TOB['TOB'] != '133') , 'TOB'] = 'OTHERS'

print(TOB['TOB'].value_counts())

MISSING    21679
131         7470
851         3433
771         2095
323         1497
OTHERS      1217
329         1139
711         1049
831          873
111          632
141          617
322          324
133          256
Name: TOB, dtype: Int64


In [54]:
#7 PAR_CD

In [55]:
grp = Fact_cols.groupby('PAR_CD')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [56]:
##  PAR_CD replacing NULLS with MISSING factor

In [57]:
Fact_cols['PAR_CD'] = Fact_cols['PAR_CD'].fillna('MISSING')

In [58]:
###### Replacing minor level+NULLS PAR_CD with 'OTHERS'

In [59]:
PAR_CD = pd.DataFrame(Fact_cols['PAR_CD'])
PAR_CD.iloc[(PAR_CD['PAR_CD'] != '7') & (PAR_CD['PAR_CD'] != 'MISSING')
                & (PAR_CD['PAR_CD'] != 'L') & (PAR_CD['PAR_CD'] != '5') & (PAR_CD['PAR_CD'] != '4')
                & (PAR_CD['PAR_CD'] != 'B') & (PAR_CD['PAR_CD'] != 'D') & (PAR_CD['PAR_CD'] != 'S')
                & (PAR_CD['PAR_CD'] != 'Y') & (PAR_CD['PAR_CD'] != 'Z') & (PAR_CD['PAR_CD'] != 'J')
                & (PAR_CD['PAR_CD'] != 'F') & (PAR_CD['PAR_CD'] != 'X') & (PAR_CD['PAR_CD'] != '$') & (PAR_CD['PAR_CD'] != 'P')
                & (PAR_CD['PAR_CD'] != 'B') & (PAR_CD['PAR_CD'] != '4') & (PAR_CD['PAR_CD'] != 'S') & (PAR_CD['PAR_CD'] != 'U'), 'PAR_CD'] = 'OTHERS'

print(PAR_CD['PAR_CD'].value_counts())

P          10085
MISSING     7250
L           5892
B           4087
Y           3078
5           2591
D           2223
X           1447
Z           1394
F           1358
OTHERS       800
4            590
7            374
S            361
$            319
J            219
U            213
Name: PAR_CD, dtype: Int64


In [60]:
#8 VCH

In [61]:
grp = Fact_cols.groupby('VCH')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [62]:
##  VCH replacing NULLS with MISSING factor

In [63]:
Fact_cols['VCH'] = Fact_cols['VCH'].fillna('MISSING')

In [64]:
###### Replacing minor level+NULLS VCH with 'OTHERS'

In [65]:
VCH = pd.DataFrame(Fact_cols['VCH'])
VCH.iloc[(VCH['VCH'] != 'MISSING') & (VCH['VCH'] != 'H') & (VCH['VCH'] != 'K')
         & (VCH['VCH'] != 'R') & (VCH['VCH'] != 'V'), 'VCH'] = 'OTHERS'

print(VCH['VCH'].value_counts())

H          33665
K           5251
V           1604
R           1213
MISSING      512
OTHERS        36
Name: VCH, dtype: Int64


In [66]:
#9 MBR_ST

In [67]:
grp = Fact_cols.groupby('MBR_ST')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [68]:
##  MBR_ST replacing NULLS with MISSING factor

In [69]:
Fact_cols['MBR_ST'] = Fact_cols['MBR_ST'].fillna('MISSING')

In [70]:
###### Replacing minor level+NULLS MBR_ST with 'OTHERS'

In [71]:
MBR_ST = pd.DataFrame(Fact_cols['MBR_ST'])
MBR_ST.iloc[(MBR_ST['MBR_ST'] != 'ME') & (MBR_ST['MBR_ST'] != 'IA') & (MBR_ST['MBR_ST'] != 'NV')
                & (MBR_ST['MBR_ST'] != 'IL') & (MBR_ST['MBR_ST'] != 'VA') & (MBR_ST['MBR_ST'] != 'OK')
                & (MBR_ST['MBR_ST'] != 'TX') & (MBR_ST['MBR_ST'] != 'TN') & (MBR_ST['MBR_ST'] != 'NM')
                & (MBR_ST['MBR_ST'] != 'FL') & (MBR_ST['MBR_ST'] != 'WV') & (MBR_ST['MBR_ST'] != 'WA')
                & (MBR_ST['MBR_ST'] != 'NC') & (MBR_ST['MBR_ST'] != 'PA') & (MBR_ST['MBR_ST'] != 'CA')
                & (MBR_ST['MBR_ST'] != 'LA') & (MBR_ST['MBR_ST'] != 'NY') & (MBR_ST['MBR_ST'] != 'AR')
                & (MBR_ST['MBR_ST'] != 'SC') & (MBR_ST['MBR_ST'] != 'MN') & (MBR_ST['MBR_ST'] != 'KS')
                & (MBR_ST['MBR_ST'] != 'OH') & (MBR_ST['MBR_ST'] != 'MO') 
                & (MBR_ST['MBR_ST'] != 'MI') & (MBR_ST['MBR_ST'] != 'WI') 
                & (MBR_ST['MBR_ST'] != 'GA') & (MBR_ST['MBR_ST'] != 'AZ') 
                & (MBR_ST['MBR_ST'] != 'IN') & (MBR_ST['MBR_ST'] != 'AL') 
                & (MBR_ST['MBR_ST'] != 'KY') & (MBR_ST['MBR_ST'] != 'MT') 
                & (MBR_ST['MBR_ST'] != 'MS') & (MBR_ST['MBR_ST'] != 'CO') & (MBR_ST['MBR_ST'] != 'NJ'), 'MBR_ST'] = 'OTHERS'

print(MBR_ST['MBR_ST'].value_counts())

IL        5429
TX        3412
FL        3045
NC        2659
LA        2416
SC        1911
OH        1671
MI        1542
GA        1539
OTHERS    1348
IN        1279
KY        1146
MS        1109
VA        1053
TN         972
WV         952
PA         875
NY         873
MN         853
MO         752
WI         728
AZ         673
AL         671
MT         601
CO         591
OK         564
NM         521
WA         510
CA         475
AR         452
KS         384
NJ         362
ME         346
IA         315
NV         252
Name: MBR_ST, dtype: Int64


In [72]:
##10 PRIM_DX_MAPPING

In [73]:
grp = Fact_cols.groupby('PRIM_DX_MAPPING')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [74]:
##  PRIM_DX_MAPPING replacing NULLS with MISSING factor

In [75]:
Fact_cols['PRIM_DX_MAPPING'] = Fact_cols['PRIM_DX_MAPPING'].fillna('MISSING')

In [76]:
###### Replacing minor level+NULLS PRIM_DX_MAPPING with 'OTHERS'

In [77]:
PRIM_DX_MAPPING = pd.DataFrame(Fact_cols['PRIM_DX_MAPPING'])
PRIM_DX_MAPPING.iloc[(PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='General symptoms and signs')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Diabetes mellitus')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Persons encountering health services for examinations')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Hypertensive diseases')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other dorsopathies')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Encounters for other specific health care')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Symptoms and signs involving the circulatory and respiratory systems')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other joint disorders')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Persons with potential health hazards related to family and personal history and certain conditions influencing health status')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other forms of heart disease')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Chronic lower respiratory diseases')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Persons with potential health hazards related to communicable diseases')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Osteoarthritis')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Disorders of lens')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Acute kidney failure and chronic kidney disease')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Spondylopathies')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Symptoms and signs involving the digestive system and abdomen')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other soft tissue disorders')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Episodic and paroxysmal disorders')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Disorders of choroid and retina')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Diseases of oral cavity and salivary glands')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other disorders of the nervous system')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Metabolic disorders')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Ischemic heart diseases')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other diseases of the urinary system')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Mood [affective] disorders')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other disorders of the skin and subcutaneous tissue')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Cerebrovascular diseases')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other diseases of intestines')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Acute upper respiratory infections')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Biomechanical lesions, not elsewhere classified"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Complications of surgical and medical care, not elsewhere classified"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Anxiety, dissociative, stress-related, somatoform and other nonpsychotic mental disorders"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Symptoms and signs involving cognition, perception, emotional state and behavior"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Diseases of esophagus, stomach and duodenum"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other diseases of the respiratory system')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Influenza and pneumonia')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Malignant neoplasms of lymphoid, hematopoietic and related tissue"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Diseases of veins, lymphatic vessels and lymph nodes, not elsewhere classified"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Diseases of arteries, arterioles and capillaries"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Injuries to the hip and thigh')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Disorders of bone density and structure')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Infections of the skin and subcutaneous tissue')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Disorders of ocular muscles, binocular movement, accommodation and refraction"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Symptoms and signs involving the nervous and musculoskeletal systems')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Injuries to the knee and lower leg')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='"Abnormal findings on diagnostic imaging and in function studies, without diagnosis"')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Symptoms and signs involving the genitourinary system')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Other bacterial diseases')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Malignant neoplasms of breast')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Malignant neoplasms of digestive organs')
& (PRIM_DX_MAPPING['PRIM_DX_MAPPING'] !='Mental and behavioral disorders due to psychoactive substance use'), 'PRIM_DX_MAPPING'] = 'OTHERS'

print(PRIM_DX_MAPPING['PRIM_DX_MAPPING'].value_counts())


OTHERS                                                                                                                           8535
General symptoms and signs                                                                                                       2528
Diabetes mellitus                                                                                                                2156
Persons encountering health services for examinations                                                                            2108
Hypertensive diseases                                                                                                            1997
Other dorsopathies                                                                                                               1538
Encounters for other specific health care                                                                                        1417
Symptoms and signs involving the circulatory and respiratory s

In [78]:
#11 SUSDUP

In [79]:
grp = Fact_cols.groupby('SUSDUP')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [80]:
Fact_cols['SUSDUP'] = Fact_cols['SUSDUP'].fillna('MISSING')

In [81]:
###### Replacing minor level+NULLS SUSDUP with 'OTHERS'

In [82]:
SUSDUP = pd.DataFrame(Fact_cols['SUSDUP'])
SUSDUP.iloc[(SUSDUP['SUSDUP'] != 'MISSING') & (SUSDUP['SUSDUP'] != 'S') & (SUSDUP['SUSDUP'] != 'R')
            & (SUSDUP['SUSDUP'] != 'M') & (SUSDUP['SUSDUP'] != 'X') & (SUSDUP['SUSDUP'] != 'V'), 'SUSDUP'] = 'OTHERS'

print(SUSDUP['SUSDUP'].value_counts())

MISSING    38541
S           1866
R            824
M            644
X            290
V            116
Name: SUSDUP, dtype: Int64


In [83]:
#11 MSL_INDICATOR

In [84]:
grp = Fact_cols.groupby('MSL_INDICATOR')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [85]:
Fact_cols['MSL_INDICATOR'] = Fact_cols['MSL_INDICATOR'].fillna('MISSING')

In [86]:
###### Replacing minor level+NULLS ADPY with 'OTHERS'

In [87]:
MSL_INDICATOR = pd.DataFrame(Fact_cols['MSL_INDICATOR'])
MSL_INDICATOR.iloc[(MSL_INDICATOR['MSL_INDICATOR'] != '1') & (MSL_INDICATOR['MSL_INDICATOR'] != '2')
                   & (MSL_INDICATOR['MSL_INDICATOR'] != 'MISSING'), 'MSL_INDICATOR'] = 'OTHERS'

print(MSL_INDICATOR['MSL_INDICATOR'].value_counts())

MISSING    40405
1           1299
2            577
Name: MSL_INDICATOR, dtype: Int64


In [88]:
#12 REFERRAL

In [89]:
grp = Fact_cols.groupby('REFERRAL')
Claim_Cnt = grp['CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [90]:
Fact_cols['REFERRAL'] = Fact_cols['REFERRAL'].fillna('MISSING')

In [91]:
###### Replacing minor level+NULLS REFERRAL with 'OTHERS'

In [92]:
REFERRAL = pd.DataFrame(Fact_cols['REFERRAL'])
REFERRAL.iloc[(REFERRAL['REFERRAL'] != 'I69110') & (REFERRAL['REFERRAL'] != 'I69814')
                   & (REFERRAL['REFERRAL'] != 'I69929') & (REFERRAL['REFERRAL'] != 'I71574')
              & (REFERRAL['REFERRAL'] != 'MISSING'), 'REFERRAL'] = 'OTHERS'

print(REFERRAL['REFERRAL'].value_counts())

MISSING    34666
OTHERS      6116
I69110       769
I69814       349
I69929       254
I71574       127
Name: REFERRAL, dtype: Int64


In [93]:
### Cbind the above categorical variable

In [94]:
Comb_Fact_Data = pd.DataFrame(pd.concat([PRCSD_AS, LOB, NTWK, PRV_SPC, TOB, PAR_CD, VCH, MBR_ST, PRIM_DX_MAPPING, SUSDUP, MSL_INDICATOR, REFERRAL, CLI_BFT],axis=1))
print(Comb_Fact_Data.columns)

Index(['PRCSD_AS', 'LOB', 'NTWK', 'PRV_SPC', 'TOB', 'PAR_CD', 'VCH', 'MBR_ST',
       'PRIM_DX_MAPPING', 'SUSDUP', 'MSL_INDICATOR', 'REFERRAL', 'CLI_BFT'],
      dtype='object')


In [95]:
######## Removing the above cols from the Fact_cols data

In [96]:
Fact_cols = Fact_cols.drop(['PRCSD_AS', 'LOB', 'NTWK', 'PRV_SPC', 'TOB', 'PAR_CD', 'VCH', 'MBR_ST', 'PRIM_DX_MAPPING', 'SUSDUP', 'MSL_INDICATOR', 'REFERRAL', 'CLI_BFT'],axis=1)

In [97]:
##### Adding the revised dataset to the Fact_cols table

In [98]:
Fact_cols = pd.concat([Fact_cols, Comb_Fact_Data],axis=1)

In [99]:
del PRCSD_AS, LOB, NTWK, PRV_SPC, TOB, PAR_CD, VCH, MBR_ST, PRIM_DX_MAPPING, SUSDUP, MSL_INDICATOR, REFERRAL, CLI_BFT, x

In [100]:
## converting dtype of categorical columns

In [101]:
Fact_cols = Fact_cols.astype({'MSL_INDICATOR':'category','REFERRAL':'category','PRCSD_AS':'category', 'LOB':'category', 'NTWK':'category', 'PRV_SPC':'category', 'TOB':'category', 'PAR_CD':'category', 'VCH':'category', 'MBR_ST':'category', 'PRIM_DX_MAPPING':'category', 'SUSDUP':'category', 'CLI_BFT':'category','AUDIT_STATUS':'category','CLAIM_NBR':'category','PROCESSED_GROUP':'category'})

In [102]:
## changing dtype of numerical columns

In [103]:
print(Num_Cols.dtypes)

CLAIM_NBR                                               Int64
LINE_ITEM_CURRENT_ORIGINAL                              Int64
LINE_ITEM_REWORK_ADD_PAY                                Int64
LINE_ITEM_OTHER_INSURANCE                               Int64
LINE_ITEM_ORIGINAL_BACKED_OUT                           Int64
LINE_ITEM_REVERSAL_LINE                                 Int64
LINE_ITEM_HISTORY_LINE                                  Int64
LINE_ITEM_OTHERS                                        Int64
CIS_CAT_MISSING                                         Int64
CIS_CAT_SERVICES_D                                      Int64
CIS_CAT_E03                                             Int64
CIS_CAT_UNIQUE_SER                                      Int64
CIS_CAT_SKILLED_NU                                      Int64
CIS_CAT_E01                                             Int64
CIS_CAT_PHYSICAL_T                                      Int64
CIS_CAT_CLINIC                                          Int64
CIS_CAT_

In [104]:
Num_Cols = Num_Cols.astype({'LINE_ITEM_CURRENT_ORIGINAL'                   :'category',
'LINE_ITEM_REWORK_ADD_PAY'                     :'category',
'LINE_ITEM_OTHER_INSURANCE'                    :'category',
'LINE_ITEM_ORIGINAL_BACKED_OUT'               :'category',
'LINE_ITEM_REVERSAL_LINE'                      :'category',
'LINE_ITEM_HISTORY_LINE'                       :'category',
'LINE_ITEM_OTHERS'                             :'category',
'CIS_CAT_MISSING'                              :'category',
'CIS_CAT_SERVICES_D'                           :'category',
'CIS_CAT_E03'                                  :'category',
'CIS_CAT_UNIQUE_SER'                           :'category',
'CIS_CAT_SKILLED_NU'                           :'category',
'CIS_CAT_E01'                                  :'category',
'CIS_CAT_PHYSICAL_T'                           :'category',
'CIS_CAT_CLINIC'                               :'category',
'CIS_CAT_OCCUPATION'                           :'category',
'CIS_CAT_CLINIC_I'                             :'category',
'CIS_CAT_NON_SURG_P'                           :'category',
'CIS_CAT_OTHERS'                               :'category',
'OVR_00'                                       :'category',
'OVR_27'                                       :'category',
'OVR_01'                                       :'category',
'OVR_37'                                       :'category',
'OVR_13'                                       :'category',
'OVR_OTHERS'                                   :'category',
'REV_MISSING'                                  :'category',
'REV_0300'                                     :'category',
'REV_0636'                                     :'category',
'REV_0301'                                     :'category',
'REV_0023'                                     :'category',
'REV_0521'                                     :'category',
'REV_0305'                                     :'category',
'REV_0250'                                     :'category',
'REV_0450'                                     :'category',
'REV_0551'                                     :'category',
'REV_0510'                                     :'category',
'REV_0421'                                     :'category',
'REV_0730'                                     :'category',
'REV_0260'                                     :'category',
'REV_0306'                                     :'category',
'REV_0320'                                     :'category',
'REV_0272'                                     :'category',
'REV_0324'                                     :'category',
'REV_0258'                                     :'category',
'REV_0983'                                     :'category',
'REV_0981'                                     :'category',
'REV_0270'                                     :'category',
'REV_0420'                                     :'category',
'REV_0307'                                     :'category',
'REV_0490'                                     :'category',
'REV_0370'                                     :'category',
'REV_0431'                                     :'category',
'REV_0360'                                     :'category',
'REV_0710'                                     :'category',
'REV_0352'                                     :'category',
'REV_0302'                                     :'category',
'REV_0424'                                     :'category',
'REV_0762'                                     :'category',
'REV_0960'                                     :'category',
'REV_0637'                                     :'category',
'REV_0761'                                     :'category',
'REV_0351'                                     :'category',
'REV_0460'                                     :'category',
'REV_0430'                                     :'category',
'REV_OTHERS'                                   :'category',
'CPT_1'                                        :'category',
'CPT_2'                                        :'category',
'CPT_3'                                        :'category',
'CPT_4'                                        :'category',
'CPT_5'                                        :'category',
'CPT_6'                                        :'category',
'CPT_7'                                        :'category',
'CPT_8'                                        :'category',
'CPT_9'                                        :'category',
'CPT_10'                                       :'category',
'CPT_11'                                       :'category',
'CPT_12'                                       :'category',
'CPT_13'                                       :'category',
'CPT_MISSING'                                  :'category',
'CPT_14'                                       :'category',
'CPT_15'                                       :'category',
'CPT_16'                                       :'category',
'CPT_OTHERS'                                   :'category',
'SFX_MISSING'                                  :'category',
'SFX_1'                                        :'category',
'SFX_2'                                        :'category',
'SFX_3'                                        :'category',
'SFX_4'                                        :'category',
'SFX_5'                                        :'category',
'SFX_6'                                        :'category',
'SFX_7'                                        :'category',
'SFX_8'                                        :'category',
'SFX_9'                                        :'category',
'SFX_10'                                       :'category',
'SFX_11'                                       :'category',
'SFX_12'                                       :'category',
'SFX_13'                                       :'category',
'SFX_14'                                       :'category',
'SFX_15'                                       :'category',
'SFX_16'                                       :'category',
'SFX_17'                                       :'category',
'SFX_18'                                       :'category',
'SFX_OTHERS'                                   :'category',
'REJ_EX_CD_MISSING'                            :'category',
'REJ_EX_CD_1'                                  :'category',
'REJ_EX_CD_2'                                  :'category',
'REJ_EX_CD_3'                                  :'category',
'REJ_EX_CD_4'                                  :'category',
'REJ_EX_CD_5'                                  :'category',
'REJ_EX_CD_6'                                  :'category',
'REJ_EX_CD_7'                                  :'category',
'REJ_EX_CD_8'                                  :'category',
'REJ_EX_CD_9'                                  :'category',
'REJ_EX_CD_10'                                 :'category',
'REJ_EX_CD_11'                                 :'category',
'REJ_EX_CD_12'                                 :'category',
'REJ_EX_CD_13'                                 :'category',
'REJ_EX_CD_14'                                 :'category',
'REJ_EX_CD_15'                                 :'category',
'REJ_EX_CD_16'                                 :'category',
'REJ_EX_CD_17'                                 :'category',
'REJ_EX_CD_18'                                 :'category',
'REJ_EX_CD_19'                                 :'category',
'REJ_EX_CD_20'                                 :'category',
'REJ_EX_CD_21'                                 :'category',
'REJ_EX_CD_22'                                 :'category',
'REJ_EX_CD_23'                                 :'category',
'REJ_EX_CD_24'                                 :'category',
'REJ_EX_CD_25'                                 :'category',
'REJ_EX_CD_26'                                 :'category',
'REJ_EX_CD_27'                                 :'category',
'REJ_EX_CD_28'                                 :'category',
'REJ_EX_CD_29'                                 :'category',
'REJ_EX_CD_30'                                 :'category',
'REJ_EX_CD_31'                                 :'category',
'REJ_EX_CD_32'                                 :'category',
'REJ_EX_CD_33'                                 :'category',
'REJ_EX_CD_34'                                 :'category',
'REJ_EX_CD_35'                                 :'category',
'REJ_EX_CD_36'                                 :'category',
'REJ_EX_CD_37'                                 :'category',
'REJ_EX_CD_38'                                 :'category',
'REJ_EX_CD_OTHERS'                             :'category',
'POT_22'                                       :'category',
'POT_11'                                       :'category',
'POT_12'                                       :'category',
'POT_23'                                       :'category',
'POT_21'                                       :'category',
'POT_24'                                       :'category',
'POT_81'                                       :'category',
'POT_31'                                       :'category',
'POT_32'                                       :'category',
'POT_99'                                       :'category',
'POT_41'                                       :'category',
'POT_20'                                       :'category',
'POT_19'                                       :'category',
'POT_02'                                       :'category',
'POT_OTHERS'                                   :'category',
'PYEE_R'                                       :'category',
'PYEE_5'                                       :'category',
'PYEE_2'                                       :'category',
'PYEE_E'                                       :'category',
'PYEE_4'                                       :'category',
'PYEE_H'                                       :'category',
'PYEE_S'                                       :'category',
'PYEE_I'                                       :'category',
'PYEE_MISSING'                                 :'category',
'PYEE_OTHERS'                                  :'category',
'CAS_CAUSE_GENERAL_ILLNESS'                    :'category',
'CAS_CAUSE_MISSING'                            :'category',
'CAS_CAUSE_ROUTINE_CARE'                       :'category',
'CAS_CAUSE_MENTAL_AND_NERVOUS'                 :'category',
'CAS_CAUSE_ACCIDENT'                           :'category',
'CAS_CAUSE_ORGAN_TRANSPLANT'                  :'category',
'CAS_CAUSE_BIRTH_CONTROL_FAMILY_PLANNING'      :'category',
'CAS_CAUSE_SERV_ARENOT_COVERED_BASED_ON_DIAG_BILLED'    :'category',
'CAS_CAUSE_ROUTINE_VISION'    :'category',
'CAS_CAUSE_DRUG_ABUSE'    :'category',
'CAS_CAUSE_OTHERS'                             :'category',
'SUSP_K'                                       :'category',
'SUSP_MISSING'                                 :'category',
'SUSP_Q'                                       :'category',
'SUSP_P'                                       :'category',
'SUSP_D'                                       :'category',
'SUSP_I'                                       :'category',
'SUSP_Y'                                       :'category',
'SUSP_OTHERS'                                  :'category'
})

In [105]:
Num_Cols.dtypes

CLAIM_NBR                                                Int64
LINE_ITEM_CURRENT_ORIGINAL                            category
LINE_ITEM_REWORK_ADD_PAY                              category
LINE_ITEM_OTHER_INSURANCE                             category
LINE_ITEM_ORIGINAL_BACKED_OUT                         category
LINE_ITEM_REVERSAL_LINE                               category
LINE_ITEM_HISTORY_LINE                                category
LINE_ITEM_OTHERS                                      category
CIS_CAT_MISSING                                       category
CIS_CAT_SERVICES_D                                    category
CIS_CAT_E03                                           category
CIS_CAT_UNIQUE_SER                                    category
CIS_CAT_SKILLED_NU                                    category
CIS_CAT_E01                                           category
CIS_CAT_PHYSICAL_T                                    category
CIS_CAT_CLINIC                                        c

In [106]:
## Combining Categorical and Numerical columns

In [107]:
Num_Cols = Num_Cols.drop(['CLAIM_NBR'],axis=1)

In [108]:
Final_Data = pd.concat([Fact_cols,Num_Cols], axis=1)
Final_Data.index = Fact_cols.index
#Final_Data = Final_Data.drop(['CLAIM_NBR'],axis=1)

In [109]:
#Final_Data[[ 'Claim Payment Accuracy', 'Claim Processing Accuracy','Financial Accuracy']] = Final_Data[[ 'Claim Payment Accuracy', 'Claim Processing Accuracy','Financial Accuracy']].fillna(value=Final_Data[[ 'Claim Payment Accuracy', 'Claim Processing Accuracy','Financial Accuracy']].mean())

In [110]:
x = Final_Data.columns
for i in x:
    print(i,":", Final_Data[i].isnull().sum()/len(Fact_cols["CLAIM_NBR"])*100)

PROCESSED_GROUP : 0.0
AUDIT_STATUS : 0.0
CLAIM_NBR : 0.0
PRCSD_AS : 0.0
LOB : 0.0
NTWK : 0.0
PRV_SPC : 0.0
TOB : 0.0
PAR_CD : 0.0
VCH : 0.0
MBR_ST : 0.0
PRIM_DX_MAPPING : 0.0
SUSDUP : 0.0
MSL_INDICATOR : 0.0
REFERRAL : 0.0
CLI_BFT : 0.0
LINE_ITEM_CURRENT_ORIGINAL : 0.0
LINE_ITEM_REWORK_ADD_PAY : 0.0
LINE_ITEM_OTHER_INSURANCE : 0.0
LINE_ITEM_ORIGINAL_BACKED_OUT : 0.0
LINE_ITEM_REVERSAL_LINE : 0.0
LINE_ITEM_HISTORY_LINE : 0.0
LINE_ITEM_OTHERS : 0.0
CIS_CAT_MISSING : 0.0
CIS_CAT_SERVICES_D : 0.0
CIS_CAT_E03 : 0.0
CIS_CAT_UNIQUE_SER : 0.0
CIS_CAT_SKILLED_NU : 0.0
CIS_CAT_E01 : 0.0
CIS_CAT_PHYSICAL_T : 0.0
CIS_CAT_CLINIC : 0.0
CIS_CAT_OCCUPATION : 0.0
CIS_CAT_CLINIC_I : 0.0
CIS_CAT_NON_SURG_P : 0.0
CIS_CAT_OTHERS : 0.0
OVR_00 : 0.0
OVR_27 : 0.0
OVR_01 : 0.0
OVR_37 : 0.0
OVR_13 : 0.0
OVR_OTHERS : 0.0
UN : 0.0
REV_MISSING : 0.0
REV_0300 : 0.0
REV_0636 : 0.0
REV_0301 : 0.0
REV_0023 : 0.0
REV_0521 : 0.0
REV_0305 : 0.0
REV_0250 : 0.0
REV_0450 : 0.0
REV_0551 : 0.0
REV_0510 : 0.0
REV_0421 : 0.0
RE

In [111]:
pickle_out = open('Final_Data_.pickle','wb')
pickle.dump(Final_Data,pickle_out)
pickle_out.close()

In [112]:
Final_Data.dtypes

PROCESSED_GROUP                                       category
AUDIT_STATUS                                          category
CLAIM_NBR                                             category
PRCSD_AS                                              category
LOB                                                   category
NTWK                                                  category
PRV_SPC                                               category
TOB                                                   category
PAR_CD                                                category
VCH                                                   category
MBR_ST                                                category
PRIM_DX_MAPPING                                       category
SUSDUP                                                category
MSL_INDICATOR                                         category
REFERRAL                                              category
CLI_BFT                                               c